In [32]:
#Test file explore_data
import os
os.chdir("D:/amazon-reviews") #Run always on the root to avoid problems


import pandas as pd
from importlib import reload
from pathlib import Path
from src.features.select_columns import split_columns  # import function
import src.text.clean_text as ct
reload(ct)

<module 'src.text.clean_text' from 'D:\\amazon-reviews\\src\\text\\clean_text.py'>

In [2]:
#Step 1: Function to load just a number of columns to properly explore data set

def preview_columns(file_path, n=200):
    """
    Loads a small preview of the dataset (first n rows) to avoids memory errors.
    
    """
    f_path = Path(file_path)
    if not f_path.exists():
        print(f"Path {f_path} not found.")
        return None

    print(f"Previewing first {n} rows from: {f_path.name}")
    df_preview = pd.read_json(f_path, lines=True, nrows=n)
    return df_preview


df_preview = preview_columns("data/raw/Musical_Instruments.json.gz")
df_preview.head()

#print(df_preview.columns)

Previewing first 200 rows from: Musical_Instruments.json.gz


,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime
0,5,90.0,False,"08 9, 2004",AXHY24HWOF184,470536454,{'Format:': ' Paperback'},Bendy,Crocheting for Dummies by Karen Manthey & Susa...,Terrific Book for Learning the Art of Crochet,1092009600
1,4,2.0,True,"04 6, 2017",A29OWR79AM796H,470536454,{'Format:': ' Hardcover'},Amazon Customer,Very helpful...,Four Stars,1491436800
2,5,NaN,True,"03 14, 2017",AUPWU27A7X5F6,470536454,{'Format:': ' Paperback'},Amazon Customer,EASY TO UNDERSTAND AND A PROMPT SERVICE TOO,Five Stars,1489449600
3,4,NaN,True,"02 14, 2017",A1N69A47D4JO6K,470536454,{'Format:': ' Paperback'},Christopher Burnett,My girlfriend use quite often,Four Stars,1487030400
4,5,NaN,True,"01 29, 2017",AHTIQUMVCGBFJ,470536454,{'Format:': ' Paperback'},Amazon Customer,Arrived as described. Very happy.,Very happy.,1485648000


In [17]:
#Step 2 Just load the columns of each file

data_dir = Path("data/raw")
files = sorted(data_dir.glob("*.json.gz")) + sorted(data_dir.glob("*.json")) #glob returns a list of all the files that end in .json

print(f"{len(files)} files found\n")

for fp in files:
    col_prev = pd.read_json(fp, lines=True, nrows=50)  
    print(f"{fp.name} has {len(col_prev.columns)} columns")
    print(sorted(col_prev.columns))
    print(" ")



3 files found

Industrial_and_Scientific.json.gz has 10 columns
['asin', 'overall', 'reviewText', 'reviewTime', 'reviewerID', 'reviewerName', 'summary', 'unixReviewTime', 'verified', 'vote']
 
Musical_Instruments.json.gz has 11 columns
['asin', 'overall', 'reviewText', 'reviewTime', 'reviewerID', 'reviewerName', 'style', 'summary', 'unixReviewTime', 'verified', 'vote']
 
Video_Games.json.gz has 10 columns
['asin', 'overall', 'reviewText', 'reviewTime', 'reviewerID', 'reviewerName', 'summary', 'unixReviewTime', 'verified', 'vote']
 


In [18]:
#Step 3: test the column separation functionn

# Use a dataset
df_test = pd.read_json("data/raw/Musical_Instruments.json.gz", lines=True, nrows=50)
cols = df_test.columns

#Original columns
print("Dataset columns:")
print(cols)


#Separated columns
print("\nFunction split_columns():")
print(split_columns(cols))


Dataset columns:
Index(['overall', 'vote', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime'],
      dtype='object')

Function split_columns():
{'text': ['reviewText', 'summary'], 'context': ['asin', 'overall', 'reviewTime', 'unixReviewTime', 'reviewerID', 'reviewerName', 'verified', 'vote', 'style']}


In [ ]:
#Step 4: Test data cleaning

test_cases = [
    "  EASY TO UNDERSTAND AND A PROMPT    SERVICE TOO	 !!!!!!!!!!! 😍😍 ",
    "💀💀 URL TeStInG: www.ilovepdf.com  i love it 🐍 so much let's test it out ",
    None,
    "  Already clean text ❌✅🎯",
    "🧠🎤Arrived as 🎮🔖 described. Very happy!."
]

for cases in test_cases:
    print(f"Original: {cases}")
    print(f"Cleaned:  {ct.clean(cases)}")
    print(" ")

Original:   EASY TO UNDERSTAND AND A PROMPT    SERVICE TOO	 !!!!!!!!!!! 😍😍 
Cleaned:  easy to understand and a prompt service too
 
Original: 💀💀 URL TeStInG: www.ilovepdf.com  i love it 🐍 so much let's test it out 
Cleaned:  url testing i love it so much let's test it out
 
Original: None
Cleaned:  
 
Original:   Already clean text ❌✅🎯
Cleaned:  already clean text
 
Original: 🧠🎤Arrived as 🎮🔖 described. Very happy!.
Cleaned:  arrived as described very happy
 
